## Agent Group Chat - News Channel Use Case (News Creation - Writer and Reviewer)

![agent-chat](./Assets/AgentChat.png)

### Installing SDK and Libraries

In [ ]:
%pip install semantic-kernel==1.28.1, azure-identity, azure-ai-projects==1.0.0b7

### Setting the Environment Variables

In [ ]:
import os

os.environ["AZURE_CLIENT_ID"] = ""
os.environ["AZURE_TENANT_ID"] = ""
os.environ["AZURE_CLIENT_SECRET"] = ""


### Creating the Azure AI Project Client

In [ ]:
from azure.identity import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MessageTextContent
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.functions import KernelFunctionFromPrompt


model = ""

project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
                           conn_str=""
)

### Naming our "Writer" and "Reviewer" Agent

In [ ]:
# Define agent names
REVIEWER_NAME = "Reviewer"
WRITER_NAME = "Writer"

### System Prompt for the "Writer" Agent

In [ ]:
writer_agent_prompt = f"""Your sole responsibility is to rewrite content according to review suggestions.
- Always apply all review directions.
- Always revise the content in its entirety without explanation.
- Never address the user. """

### Creating the "Writer" Agent

In [ ]:
# [START create_agent] Creating Azure AI Agent Service Agent
writer_agent_definition = await project_client.agents.create_agent(  #add get_agent(assistant_id="") for using already created agent
        model=model,
        name=WRITER_NAME,
        instructions=writer_agent_prompt,
    )

# 2. Create a Semantic Kernel agent based on the agent definition
agent_writer = AzureAIAgent(
        client=project_client,
        definition=writer_agent_definition,
    )

### System Prompt for the "Reviewer" Agent

In [ ]:
reviewer_agent_prompt = f"""You are a reviewer agent. Your task is to evaluate a short article (~100 words) written by another agent.
Review it based on the following criteria:

1. Clarity – Is the message clear and easy to follow?
2. Accuracy – Does the content appear factually correct?
3. Engagement – Is it interesting and appropriate for a general audience?
4. Language – Are there grammar or style issues?

Provide:

1. A brief evaluation (2–3 lines).
2. A score between 1–10 reflecting the overall quality of the article.
3. If the score is 8 or below, provide 1–3 suggestions to improve the article.
4. If the score is above 8, simply state: "The article is good to go."

Do not ask for further refinement unless the score is 8 or lower."""

### Creating the "Reviewer" Agent

In [ ]:
# [START create_agent] Creating Azure AI Agent Service Agent
reviewer_agent_definition = await project_client.agents.create_agent(  #add get_agent(assistant_id="") for using already created agent
        model=model,
        name=REVIEWER_NAME,
        instructions=reviewer_agent_prompt
    )

# 2. Create a Semantic Kernel agent based on the agent definition
agent_reviewer = AzureAIAgent(
        client=project_client,
        definition=reviewer_agent_definition,
    )

### Creating the Kernel

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import KernelArguments
import os
import asyncio
import time

kernel = Kernel()

# Load environment variables from .env file
api_key = ""
deployment_name = ""
endpoint = ""

print(f"API Key: {api_key}")
print(f"Deployment Name: {deployment_name}")
print(f"Endpoint: {endpoint}")

service_id = "service1"
kernel.add_service(
    AzureChatCompletion(service_id=service_id,
                        api_key=api_key,
                        deployment_name=deployment_name,
                        endpoint = endpoint
    )
)

### Defining our "Agent Selection Function"

In [ ]:
# Define a selection function to determine which agent should take the next turn.
selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""
Examine the provided RESPONSE and choose the next participant.
State only the name of the chosen participant without explanation.
Never choose the participant named in the RESPONSE.

Choose only from these participants:
- {REVIEWER_NAME}
- {WRITER_NAME}

Rules:
- If RESPONSE is user input, it is {WRITER_NAME}'s turn.
- If RESPONSE is by {REVIEWER_NAME}, it is {WRITER_NAME}'s turn.
- If RESPONSE is by {WRITER_NAME}, it is {REVIEWER_NAME}'s turn.

RESPONSE:
{{{{$lastmessage}}}}
""",
)

### Defining our "Conversation Termination Function"

In [ ]:
# Define a termination function where the reviewer signals completion with "yes".
termination_keyword = "yes"

termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt=f"""
Examine the RESPONSE and determine whether the content has been deemed satisfactory.
The content is deemed satisfactory if the reviewer has given a score of 8 or higher and stated "The article is good to go."
If the content is satisfactory, respond with a single word without explanation: {termination_keyword}.


RESPONSE:
{{{{$lastmessage}}}}
""",
    )

### Creating our Agent "Group Chat"

In [ ]:
history_reducer = ChatHistoryTruncationReducer(target_count=5)

# Create the AgentGroupChat with selection and termination strategies.
chat = AgentGroupChat(
        agents=[agent_reviewer, agent_writer],
        selection_strategy=KernelFunctionSelectionStrategy(
            initial_agent=agent_writer,
            function=selection_function,
            kernel=kernel,
            result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else WRITER_NAME,
            history_variable_name="lastmessage",
            history_reducer=history_reducer,
        ),
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[agent_reviewer],
            function=termination_function,
            kernel=kernel,
            result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
            history_variable_name="lastmessage",
            maximum_iterations=10,
            history_reducer=history_reducer,
        ),
    )

### Running our Agent Group Chat

In [ ]:
print(
        "Ready! Type your input, or 'exit' to quit, 'reset' to restart the conversation. "
    )

is_complete = False
while not is_complete:
        print()
        user_input = input("User > ").strip()
        if not user_input:
            continue

        if user_input.lower() == "exit":
            is_complete = True
            break

        if user_input.lower() == "reset":
            await chat.reset()
            print("[Conversation has been reset]")
            continue
        
        # Add the current user_input to the chat
        await chat.add_chat_message(message=user_input)

        try:
            async for response in chat.invoke():
                if response is None or not response.name:
                    continue
                print()
                print(f"# {response.name.upper()}:\n{response.content}")
        except Exception as e:
            print(f"Error during chat invocation: {e}")

        # Reset the chat's complete flag for the new conversation round.
        chat.is_complete = False